**download the dataset from kaggle** 

In [1]:
# Install Kaggle strictly on the remote server
!pip install -q kaggle torch torchvision matplotlib scikit-learn

In [2]:
import os

In [ ]:
#access from .env file
os.environ['KAGGLE_USERNAME'] = os.getenv("KAGGLE_USERNAME")
os.environ['KAGGLE_KEY'] = os.getenv("KAGGLE_KEY")

Kaggle Username: tekleeyesusmunye
Kaggle Key: KGAT_44361b39e21b62fdfd7d135978805a16


In [4]:
# Download Dataset (Chest X-Ray Images (Pneumonia))
print("Downloading dataset...")
!kaggle datasets download -d paultimothymooney/chest-xray-pneumonia --unzip -p /content/dataset

print("Dataset downloaded and unzipped!")

Dataset URL: https://www.kaggle.com/datasets/paultimothymooney/chest-xray-pneumonia
License(s): other
 99% 2.27G/2.29G [00:19<00:00, 195MB/s] 
100% 2.29G/2.29G [00:19<00:00, 128MB/s]
Dataset downloaded and unzipped!


Data Preprocessing

In [5]:
import torch
import torchvision
from torchvision import transforms, datasets
from torch.utils.data import DataLoader

# Define specific constants
IMG_SIZE = 224
BATCH_SIZE = 32

# Augmentation for robustness (handling imperfect 'photos of X-rays')
train_transforms = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.RandomRotation(10),  
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=0.1, contrast=0.1),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

val_transforms = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Directories (Kaggle dataset structure)
DATA_DIR = "/content/dataset/chest_xray"
train_dir = os.path.join(DATA_DIR, "train")
test_dir = os.path.join(DATA_DIR, "test")

# Load Data
train_data = datasets.ImageFolder(train_dir, transform=train_transforms)
test_data = datasets.ImageFolder(test_dir, transform=val_transforms)

train_loader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_data, batch_size=BATCH_SIZE, shuffle=False)

class_names = train_data.classes
print(f"Classes found: {class_names}") # Should be ['NORMAL', 'PNEUMONIA']
print(f"Training images: {len(train_data)}")

Classes found: ['NORMAL', 'PNEUMONIA']
Training images: 5216


define the model

In [6]:
import torch.nn as nn
from torchvision import models

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Training on: {device}")

model = models.resnet50(pretrained=True)

# Freeze early layers to retain learned features
for param in model.parameters():
    param.requires_grad = False

# Modify the final layer for binary classification
num_ftrs = model.fc.in_features
model.fc = nn.Sequential(
    nn.Linear(num_ftrs, 512),
    nn.ReLU(),
    nn.Dropout(0.3),
    nn.Linear(512, 2) # Output: [Normal_Score, Pneumonia_Score]
)

model = model.to(device)

# Loss and Optimizer
criterion = nn.CrossEntropyLoss()
# Only optimize the parameters of the new final layer
optimizer = torch.optim.Adam(model.fc.parameters(), lr=0.001)

Training on: cuda


/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth


100%|██████████| 97.8M/97.8M [00:00<00:00, 217MB/s]


In [7]:
def train_model(model, criterion, optimizer, num_epochs=5):
    for epoch in range(num_epochs):
        print(f"Epoch {epoch+1}/{num_epochs}")
        print("-" * 10)

        model.train()
        running_loss = 0.0
        correct = 0
        total = 0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()

            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        epoch_loss = running_loss / len(train_data)
        epoch_acc = correct / total

        print(f"Train Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}")

    return model

# Train for 3-5 epochs to balance speed and performance
trained_model = train_model(model, criterion, optimizer, num_epochs=5)

Epoch 1/5
----------
Train Loss: 0.2935 Acc: 0.8750
Epoch 2/5
----------
Train Loss: 0.1812 Acc: 0.9319
Epoch 3/5
----------
Train Loss: 0.1776 Acc: 0.9277
Epoch 4/5
----------
Train Loss: 0.1700 Acc: 0.9310
Epoch 5/5
----------
Train Loss: 0.1583 Acc: 0.9362


In [34]:
# Cell 5: Save and Download
from google.colab import files

# 1. Save standard PyTorch model (for FastAPI Backend)
torch.save(trained_model.state_dict(), "mediscan_resnet50.pt")
print("Model saved to Remote VM.")

# 2. Trigger Download to Local Machine
# This usually pops up a file save dialog in your browser/VS Code
files.download('mediscan_resnet50.pt')

Model saved to Remote VM.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>